In [12]:
import os
import s3fs
import pandas as pd
import torch
import io
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchvision.io import read_image
from PIL import Image

os.chdir("../")

from src.core.configs import settings

In [4]:
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': settings.s3_endpointUrl},
    key =settings.s3_key,
    secret = settings.s3_secret,
    token = settings.s3_token
)

In [5]:
pd.read_csv(fs.open(f"{settings.s3_prefix}_classes.csv"))

,filename,with_mask,without_mask
0,26-with-mask_jpg.rf.fb2a4fd14461ba4e7d981f51d9...,1,0
1,66_jpg.rf.fb48333563aba0cd06afba343f625d98.jpg,0,1
2,153-with-mask_jpg.rf.fb584ce29ce6d9a3123a04d80...,1,0
3,29-with-mask_jpg.rf.fb9ecac8d3dc335a43b79a9438...,1,0
4,augmented_image_315_jpg.rf.fba9c25c6675c5a039f...,1,0
...,...,...,...
1274,417_jpg.rf.64bed386d653f324066a9377a9dbbff8.jpg,0,1
1275,augmented_image_117_jpg.rf.6433692ef34560a302c...,0,1
1276,290_jpg.rf.65fd4f514fa0882844857ee5b4ff4880.jpg,0,1
1277,305-with-mask_jpg.rf.65c190086fbfb468313020747...,1,0


In [16]:


class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = f'{self.img_dir}{self.img_labels.iloc[idx, 0]}'
        image = Image.open(io.BytesIO(fs.open(img_path).read()))
        image = ToTensor()(image)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [20]:
data=CustomImageDataset(fs.open(f"{settings.s3_prefix}_classes.csv"),settings.s3_prefix )

In [21]:
data[0]

(tensor([[[0.8314, 0.8353, 0.8431,  ..., 0.9176, 0.9176, 0.9176],
          [0.8431, 0.8471, 0.8510,  ..., 0.9176, 0.9176, 0.9176],
          [0.8549, 0.8549, 0.8588,  ..., 0.9137, 0.9137, 0.9137],
          ...,
          [0.2157, 0.2275, 0.2471,  ..., 0.3059, 0.3098, 0.3137],
          [0.1961, 0.2196, 0.2510,  ..., 0.2902, 0.2941, 0.2980],
          [0.1843, 0.2000, 0.2275,  ..., 0.2667, 0.2706, 0.2745]],
 
         [[0.7216, 0.7255, 0.7333,  ..., 0.8353, 0.8353, 0.8353],
          [0.7333, 0.7373, 0.7412,  ..., 0.8353, 0.8353, 0.8353],
          [0.7451, 0.7451, 0.7490,  ..., 0.8314, 0.8314, 0.8314],
          ...,
          [0.1529, 0.1647, 0.1843,  ..., 0.2000, 0.2078, 0.2118],
          [0.1137, 0.1451, 0.1686,  ..., 0.1843, 0.1922, 0.1961],
          [0.0941, 0.1176, 0.1373,  ..., 0.1608, 0.1686, 0.1725]],
 
         [[0.7098, 0.7137, 0.7216,  ..., 0.8549, 0.8549, 0.8549],
          [0.7216, 0.7255, 0.7294,  ..., 0.8549, 0.8549, 0.8549],
          [0.7333, 0.7333, 0.7373,  ...,